In [316]:
import pandas as pd
from dateutil.relativedelta import *
from sklearn.preprocessing import OneHotEncoder

In [317]:
base_path = './Data/'
constSta = pd.read_csv(base_path+'ConstructorStandings.csv')
driSta = pd.read_csv(base_path+'DriverStandings.csv')
races = pd.read_csv(base_path+'Races.csv')
results = pd.read_csv(base_path+'Results.csv')
weaInfo = pd.read_csv(base_path+'WeatherInfo.csv')
quaResults = pd.read_csv(base_path+'QualifyingResults.csv')

Data Cleaning

In [318]:
constSta = constSta[(constSta['season'] >= 2014)]
constSta = constSta.reset_index(drop=True)

driSta = driSta[(driSta['season'] >= 2014)]
driSta = driSta.reset_index(drop=True)

races = races[(races['season'] >= 2014)]
races = races.reset_index(drop=True)

results = results[(results['season'] >= 2014)]
results = results.reset_index(drop=True)

weaInfo = weaInfo[(weaInfo['season'] >= 2014)]
weaInfo = weaInfo.reset_index(drop=True)

quaResults = quaResults[(quaResults['season'] >= 2014)]
quaResults = quaResults.reset_index(drop=True)

In [319]:
constSta.to_csv("Data/ConstructorStandings2014.csv")
driSta.to_csv("Data/DriverStandings2014.csv")
races.to_csv("Data/Races2014.csv")
results.to_csv("Data/Results2014.csv")
weaInfo.to_csv("Data/WeatherInfo2014.csv")
quaResults.to_csv("Data/QualifyingResults2014.csv")

In [320]:
constSta = constSta.drop('Unnamed: 0', axis=1)
driSta = driSta.drop('Unnamed: 0', axis=1)
races = races.drop('Unnamed: 0', axis=1)
results = results.drop('Unnamed: 0', axis=1)
weaInfo = weaInfo.drop('Unnamed: 0', axis=1)
quaResults = quaResults.drop('Unnamed: 0', axis=1)

Races

In [321]:
races = races.drop(['url', 'lat', 'long', 'country'], axis=1)

races['date'] = pd.to_datetime(races.date)

In [322]:
races.head()

,season,round,circuit_id,date
0,2014,1,albert_park,2014-03-16
1,2014,2,sepang,2014-03-30
2,2014,3,bahrain,2014-04-06
3,2014,4,shanghai,2014-04-20
4,2014,5,catalunya,2014-05-11


Results

In [323]:
results.isnull().sum()

season              0
round               0
circuit_id          0
driver              0
date_of_birth       0
nationality         0
constructor         0
grid                0
time             1912
status              0
points              3
podium              0
dtype: int64

In [324]:
results = results.drop('time', axis=1)


In [325]:
nullRows = results[results.isnull().any(axis=1)]
nullRows

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,status,points,podium
3047,2021,12,spa,max_verstappen,1997-09-30,Dutch,red_bull,1,Finished,NaN,1
3049,2021,12,spa,hamilton,1985-01-07,British,mercedes,3,Finished,NaN,3
3056,2021,12,spa,sainz,1994-09-01,Spanish,ferrari,11,Finished,NaN,10


In [326]:
results.loc[3047, 'points'] = 12.5
results.loc[3049, 'points'] = 7.5
results.loc[3056, 'points'] = 0.5

In [327]:
statusDict = {'Finished': ['Finished', '+1 Lap', '+2 Laps', '+8 Laps', '+3 Laps', '+4 Laps', '+6 Laps', '+5 Laps'],
              'Mechanical Issue': ['ERS', 'Oil pressure', 'Engine', 'Technical', 'Gearbox', 'Electrical', 'Power Unit', 
                                   'Brakes', 'Clutch', 'Retired', 'Exhaust', 'Mechanical', 'Turbo', 'Rear wing', 'Drivetrain', 
                                   'Suspension', 'Oil leak', 'Water leak', 'Water pressure', 'Electronics', 'Wheel', 'Power loss',
                                   'Fuel system', 'Transmission', 'Front wing', 'Tyre', 'Throttle', 'Brake duct', 'Hydraulics',
                                   'Battery', 'Puncture', 'Overheating', 'Wheel nut', 'Vibrations', 'Driveshaft', 'Fuel pressure',
                                   'Seat', 'Spark plugs', 'Steering', 'Damage', 'Out of fuel', 'Debris', 'Radiator', 
                                   'Cooling system', 'Water pump', 'Fuel leak', 'Fuel pump', 'Undertray', 'Differential'],
              'Incident': ['Collision', 'Disqualified','Withdrew', 'Accident', 'Collision damage', 'Spun off', 'Excluded'],
              'Illness': ['Illness']
    }

for key, value in statusDict.items():
        results['status'] = results['status'].replace(value, key)
# 'Excluded', 'Illness'

In [328]:
results['status'].unique()

array(['Finished', 'Mechanical Issue', 'Incident', 'Illness'],
      dtype=object)

In [345]:
results.tail()

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,status,points,podium
3762,2023,3,albert_park,sargeant,2000-12-31,American,williams,18,Incident,0.0,16
3763,2023,3,albert_park,kevin_magnussen,1992-10-05,Danish,haas,14,Incident,0.0,17
3764,2023,3,albert_park,russell,1998-02-15,British,mercedes,2,Mechanical Issue,0.0,18
3765,2023,3,albert_park,albon,1996-03-23,Thai,williams,8,Incident,0.0,19
3766,2023,3,albert_park,leclerc,1997-10-16,Monegasque,ferrari,7,Incident,0.0,20


In [350]:
untested = results[(results['season'] == 2023)]
untested.to_csv('Data/Untested.csv', index=False)

Constructor Standings

In [330]:
constSta.head()

,season,round,constructor,constructor_points,constructor_wins,constructor_standings_pos
0,2014,1,mclaren,0.0,0.0,0.0
1,2014,1,mercedes,0.0,0.0,0.0
2,2014,1,ferrari,0.0,0.0,0.0
3,2014,1,williams,0.0,0.0,0.0
4,2014,1,force_india,0.0,0.0,0.0


Driver Standings

In [331]:
driSta.head()

,season,round,driver,driver_points,driver_wins,driver_standings_pos
0,2014,1,rosberg,0.0,0.0,0.0
1,2014,1,kevin_magnussen,0.0,0.0,0.0
2,2014,1,button,0.0,0.0,0.0
3,2014,1,alonso,0.0,0.0,0.0
4,2014,1,bottas,0.0,0.0,0.0


Qualifying Results

In [332]:
quaResults.isnull().sum()


grid                 0
driver_name          0
car                  0
qualifying_time    241
season               0
round                0
dtype: int64

In [333]:
nullRows = quaResults[quaResults.isnull().any(axis=1)]
nullRows

,grid,driver_name,car,qualifying_time,season,round
20,21,Pastor Maldonado MAL,Lotus Renault,NaN,2014,1
86,22,Pastor Maldonado MAL,Lotus Renault,NaN,2014,4
100,14,Kevin Magnussen MAG,McLaren Mercedes,NaN,2014,5
101,15,Sebastian Vettel VET,Red Bull Racing Renault,NaN,2014,5
107,21,Jean-Eric Vergne VER,STR Renault,NaN,2014,5
...,...,...,...,...,...,...
3640,16,Esteban Ocon OCO,Alpine Renault,NaN,2022,21
3641,17,Fernando Alonso ALO,Alpine Renault,NaN,2022,21
3642,18,Nicholas Latifi LAT,Williams Mercedes,NaN,2022,21
3643,19,Alexander Albon ALB,Williams Mercedes,NaN,2022,21


In [334]:
quaResults = quaResults.drop('driver_name', axis=1)
quaResults = quaResults.drop('car', axis=1)

In [335]:
quaResults.head()

,grid,qualifying_time,season,round
0,1,1:44.231,2014,1
1,2,1:44.548,2014,1
2,3,1:44.595,2014,1
3,4,1:45.745,2014,1
4,5,1:45.819,2014,1


Weather Info

In [336]:
weaInfo = weaInfo.drop('weather', axis=1)

In [337]:
weaInfo.head()

,season,round,circuit_id,weather_warm,weather_cold,weather_wet,weather_cloudy
0,2014,1,albert_park,0,0,0,0
1,2014,2,sepang,0,0,0,0
2,2014,3,bahrain,1,0,0,0
3,2014,4,shanghai,1,0,0,0
4,2014,5,catalunya,1,0,0,0


Data Perparation

In [351]:
finalDf = pd.merge(races, results, how = 'inner', on=['season', 'round', 'circuit_id'])
finalDf = pd.merge(finalDf, weaInfo, how = 'inner', on=['season', 'round', 'circuit_id'])
finalDf = pd.merge(finalDf, driSta, how = 'left', on=['season', 'round', 'driver'])
finalDf = pd.merge(finalDf, constSta, how = 'left', on=['season', 'round', 'constructor'])
finalDf = pd.merge(finalDf, quaResults, how = 'inner', on=['season', 'round', 'grid'])

finalDf.tail()

,season,round,circuit_id,date,driver,date_of_birth,nationality,constructor,grid,status,...,weather_cold,weather_wet,weather_cloudy,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time
3626,2022,22,yas_marina,2022-11-20,mick_schumacher,1999-03-22,German,haas,12,Finished,...,0,0,0,12.0,0.0,16.0,37.0,0.0,8.0,1:25.225
3627,2022,22,yas_marina,2022-11-20,kevin_magnussen,1992-10-05,Danish,haas,16,Finished,...,0,0,0,25.0,0.0,13.0,37.0,0.0,8.0,1:25.834
3628,2022,22,yas_marina,2022-11-20,hamilton,1985-01-07,British,mercedes,5,Mechanical Issue,...,0,0,0,240.0,0.0,5.0,505.0,1.0,3.0,1:24.508
3629,2022,22,yas_marina,2022-11-20,latifi,1995-06-29,Canadian,williams,20,Incident,...,0,0,0,2.0,0.0,20.0,8.0,0.0,10.0,1:26.054
3630,2022,22,yas_marina,2022-11-20,alonso,1981-07-29,Spanish,alpine,10,Mechanical Issue,...,0,0,0,81.0,0.0,9.0,167.0,0.0,4.0,1:25.096


In [339]:
finalDf['date_of_birth'] = pd.to_datetime(finalDf.date_of_birth)
finalDf['date'] = pd.to_datetime(finalDf.date)


finalDf['driver_age'] = finalDf.apply(lambda x: 
                                        relativedelta(x['date'], x['date_of_birth']).years, axis=1)

finalDf.drop(['date_of_birth'], axis = 1, inplace = True)
finalDf.drop(['date'], axis = 1, inplace = True)

In [340]:
# finalDf['constructor'].unique()

teamsDict = {'mercedes': ['mercedes'],
             'ferrari': ['ferrari'],
             'mclaren': ['mclaren'],
             'red_bull': ['red_bull'],
             'williams': ['williams'],
             'haas': ['haas'],
             'aston_martin': ['aston_martin', 'force_india', 'racing_point'],
             'alpine': ['alpine', 'renault', 'lotus_f1'],
             'alphatauri': ['alphatauri', 'toro_rosso'], 
             'alfa': ['alfa', 'sauber'],
             'manor': ['manor', 'marussia']
            }

for key, value in teamsDict.items():
        finalDf['constructor'] = finalDf['constructor'].replace(value, key)


In [352]:
finalDf['qualifying_time'] = finalDf.qualifying_time.map(lambda x: 0 if str(x) == '00.000' 
                             else(float(str(x).split(':')[1]) + 
                                  (60 * float(str(x).split(':')[0])) if x != 0 else 0))
final_df = finalDf[finalDf['qualifying_time'] != 0]
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round']).qualifying_time.diff()
final_df['qualifying_time'] = final_df.groupby(['season', 
                                                'round']).qualifying_time_diff.cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)

IndexError: list index out of range

In [342]:
finalDf.to_csv("Data/TempFinal.csv", index = False)

qualifying time handle
status
starting grid

In [343]:
ohe = OneHotEncoder(sparse=False)

df_ohe = finalDf

categories = ['circuit_id', 'nationality', 'constructor', 'status'] 
ohe.fit(df_ohe[['circuit_id', 'nationality', 'constructor', 'status']])

temp_df = pd.DataFrame(data=ohe.transform(df_ohe[['circuit_id', 'nationality', 'constructor', 'status']]), columns=ohe.get_feature_names_out())
df_ohe.drop(columns=['circuit_id', 'nationality', 'constructor', 'status'], axis=1, inplace=True)
df_ohe = pd.concat([df_ohe.reset_index(drop=True), temp_df], axis=1)



In [344]:
df_ohe.to_csv("Data/Final.csv", index = False)